In [1]:
# Import statements
import numpy as np
import plotly.graph_objs as go
import pandas as pd
from dotenv import dotenv_values
import plotly.graph_objects as go
SECRETS = dotenv_values("../envs/.env")


In [2]:
TRAIN_DATA_PATH = SECRETS['TRAIN_DATA_PATH']
TEST_DATA_PATH = SECRETS['TEST_DATA_PATH']
PULL_UP_STRENGTH, PULL_DOWN_STRENGTH = list(map(int, SECRETS['BETA'].split(':')))
NUM_SAMPLES = int(SECRETS['NUM_SAMPLES'])

In [3]:
df_inputs = pd.read_csv(TRAIN_DATA_PATH+"\\traindata_before_processing.csv")
df_delays = pd.read_excel(TRAIN_DATA_PATH+"\\delayvalues2.xlsx")

def convert_to_nanoseconds(delay):
    if delay.endswith('p'):
        # Convert picoseconds to nanoseconds (1 picosecond = 1e-3 nanoseconds)
        return float(delay[:-1]) * 1e-3
    elif delay.endswith('n'):
        # Nanoseconds remain unchanged
        return float(delay[:-1])
    else:
        raise ValueError("Unknown time unit")
    

df_delays['delay'] = df_delays['delay'].apply(convert_to_nanoseconds)

# df_delays['delay'] = df_delays['delay'].str.replace('p', '').astype(int) / 1000


In [4]:
# Helix equation
k, load_cap, input_cap, delay = df_inputs['k'], df_inputs['load_capacitance'], df_inputs['input_capacitance'], df_delays['delay']

fig = go.Figure(data=[go.Scatter3d(
    x=k,
    y=load_cap,
    z=input_cap,
    mode='markers',
    marker=dict(
        size=12,
        color=delay,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(  scene = dict(
                    xaxis_title='k',
                    yaxis_title='load capacitance',
                    zaxis_title='input capacitance'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
fig.show()

In [6]:
np.corrcoef(delay, k), np.corrcoef(delay, load_cap), np.corrcoef(delay, input_cap)

(array([[ 1.       , -0.0123614],
        [-0.0123614,  1.       ]]),
 array([[1.        , 0.68095053],
        [0.68095053, 1.        ]]),
 array([[1.        , 0.73133256],
        [0.73133256, 1.        ]]))

In [7]:
np.corrcoef(k, load_cap)

array([[ 1.        , -0.01182342],
       [-0.01182342,  1.        ]])